In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd

C:\Users\dougl\anaconda3\envs\dev\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\dougl\anaconda3\envs\dev\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\dougl\anaconda3\envs\dev\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\dougl\anaconda3\envs\dev\lib\site-packages\tensorflow\python\framework

In [2]:
base = pd.read_csv('test.csv')

In [3]:
x = base.drop('Forma de onda', axis=1) # atributos previsores
y = base.iloc[:,-1] # pegando a ultima coluna para saber qual a classe

In [4]:
from sklearn.preprocessing import StandardScaler
scaler_x = StandardScaler()
x = scaler_x.fit_transform(x)
x.shape

(3000, 1000)

In [5]:
from sklearn.preprocessing import OneHotEncoder
onehot = OneHotEncoder(categories = 'auto')
y.shape

(3000,)

In [6]:
y = y.values.reshape(-1,1)
y.shape

(3000, 1)

In [7]:
y = onehot.fit_transform(y).toarray()
y

array([[1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       ...,
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 0., 1.]])

In [8]:
from sklearn.model_selection import train_test_split
x_treinamento, x_teste, y_treinamento, y_teste = train_test_split(x, y, test_size = 0.3)

In [9]:
x.shape, x_teste.shape, y.shape, y_teste.shape

((3000, 1000), (900, 1000), (3000, 3), (900, 3))

In [10]:
neuronios_entrada = x.shape[1]
neuronios_entrada

1000

In [11]:
neuronios_oculta = int((neuronios_entrada + y.shape[1])/2)
neuronios_oculta

501

In [12]:
neuronios_saida = y.shape[1]
neuronios_saida

3

In [13]:
w = {'oculta': tf.Variable(tf.random_normal([neuronios_entrada, neuronios_oculta])),
     'saida': tf.Variable(tf.random_normal([neuronios_oculta, neuronios_saida]))}
w

{'oculta': <tf.Variable 'Variable:0' shape=(1000, 501) dtype=float32_ref>,
 'saida': <tf.Variable 'Variable_1:0' shape=(501, 3) dtype=float32_ref>}

In [14]:
b = {'oculta': tf.Variable(tf.random_normal([neuronios_oculta])),
     'saida': tf.Variable(tf.random_normal([neuronios_saida]))}
b

{'oculta': <tf.Variable 'Variable_2:0' shape=(501,) dtype=float32_ref>,
 'saida': <tf.Variable 'Variable_3:0' shape=(3,) dtype=float32_ref>}

In [15]:
xph = tf.placeholder('float', [None, neuronios_entrada])
yph = tf.placeholder('float', [None, neuronios_saida])

In [16]:
def mlp(x, w, bias):
    camada_oculta = tf.add(tf.matmul(x, w['oculta']), b['oculta'])
    camada_oculta_ativacao = tf.nn.relu(camada_oculta)
    camada_saida = tf.add(tf.matmul(camada_oculta_ativacao, w['saida']), b['saida'])
    return camada_saida

In [17]:
modelo = mlp(xph, w, b)

In [18]:
erro = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits = modelo, labels = yph))
otimizador = tf.train.AdamOptimizer(learning_rate = 0.01).minimize(erro)

In [19]:
batch_size = 8
batch_total = int(len(x_treinamento) / batch_size)
batch_total

262

In [20]:
x_batches = np.array_split(x_treinamento, batch_total)

In [ ]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for epoca in range(3000):
        erro_medio = 0.0
        batch_total = int(len(x_treinamento) / batch_size)
        x_batches = np.array_split(x_treinamento, batch_total)
        y_batches = np.array_split(y_treinamento, batch_total)
        for i in range(batch_total):
            x_batch, y_batch = x_batches[i], y_batches[i]
            _, custo = sess.run([otimizador, erro], feed_dict = {xph: x_batch, yph: y_batch})
            erro_medio += custo / batch_total
        if epoca % 500 == 0:
            print('Época: ' + str(epoca+1) + ' erro: '+ str(erro_medio))
    w_final, b_final = sess.run([w,b])

Época: 1 erro: 85.09026367124024


In [ ]:
previsoes_teste = mlp(xph, w_final, b_final)

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    r1 = sess.run(previsoes_teste, feed_dict = {xph: x_teste})
    r2 = sess.run(tf.nn.softmax(r1))
    r3 = sess.run(tf.argmax(r2, 1))

In [ ]:
y_teste2 = np.argmax(y_teste, 1)

In [ ]:
from sklearn.metrics import accuracy_score
taxa_acerto = accuracy_score(y_teste2, r3)
taxa_acert

o